# Distributed training with Amazon SageMaker XGBoost script mode
_**Regression with Amazon SageMaker XGBoost algorithm**_

---

## Contents
1. [Introduction](#Introduction)
2. [Setup](#Setup)
  1. [Data Ingestion](#Data-ingestion)
  2. [Preparing the dataset](#Preparing-the-dataset)
3. [Training the XGBoost model](#Training-the-XGBoost-model)
3. [Deploying the XGBoost model](#Deploying-the-XGBoost-model)
4. [Orchestration](#Orchestration)
---

## Introduction

This notebook demonstrates the use of Amazon SageMaker XGBoost to train and host a regression model. [XGBoost (eXtreme Gradient Boosting)](https://xgboost.readthedocs.io) is a popular and efficient machine learning algorithm used for regression and classification tasks on tabular datasets. It implements a technique know as gradient boosting on trees, and performs remarkably well in machine learning competitions, and gets a lot of attention from customers. 

We use the [Abalone data](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression.html), originally from the [UCI data repository](https://archive.ics.uci.edu/ml/datasets/abalone). More details about the original dataset can be found [here](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names). Age of abalone is to be predicted from eight physical measurements.  

---
## Setup

This notebook was tested in Amazon SageMaker Studio on a ml.t3.medium instance with Python 3 (Data Science) kernel.

Let's start by specifying:
1. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
1. The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with the appropriate full IAM role arn string(s).

In [ ]:
%%time

import os
import boto3
import re
import sagemaker

# Get a SageMaker-compatible role used by this Notebook Instance.
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

### update below values appropriately ###
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/xgboost-dist-script'

print(region)

### Data ingestion

Next, we read the dataset from the existing repository into memory, for preprocessing prior to training. This processing could be done *in situ* by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

In [ ]:
%%time

import pyarrow
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file

s3 = boto3.client("s3")
# Download the dataset and load into a pandas dataframe
FILE_NAME = 'abalone.csv'
s3.download_file("sagemaker-sample-files", f"datasets/tabular/uci_abalone/{FILE_NAME}", FILE_NAME)

feature_names=['Sex', 
               'Length', 
               'Diameter', 
               'Height', 
               'Whole weight', 
               'Shucked weight', 
               'Viscera weight', 
               'Shell weight', 
               'Rings']

data = pd.read_csv(FILE_NAME, 
                   header=None, 
                   names=feature_names)
data["Sex"] = data["Sex"].astype("category").cat.codes

data.head()

### Preparing the dataset

Following methods split the data into train/validation datasets and upload files to S3.

In [ ]:
# SageMaker XGBoost has the convention of label in the first column
data = data[feature_names[-1:] + feature_names[:-1]]
data.head()

In [ ]:
# Split the downloaded data into train/test dataframes
train, validation = np.split(data.sample(frac=1), [int(.8*len(data))])
train_0, train_1 = np.split(train.sample(frac=1), [int(.5*len(train))])

train_0.to_parquet('abalone_train_0.parquet')
train_1.to_parquet('abalone_train_1.parquet')
validation.to_parquet('abalone_validation.parquet')

In [ ]:
%%time

sagemaker.Session().upload_data('abalone_train_0.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'train')

sagemaker.Session().upload_data('abalone_train_1.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'train')

sagemaker.Session().upload_data('abalone_validation.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'validation')

## Create a XGBoost script to train with 

SageMaker can now run an XGboost script using the XGBoost estimator. When executed on SageMaker a number of helpful environment variables are available to access properties of the training environment, such as:

- `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to. Any artifacts saved in this folder are uploaded to S3 for model hosting after the training job completes.
- `SM_OUTPUT_DIR`: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

Supposing two input channels, 'train' and 'validation', were used in the call to the XGBoost estimator's fit() method, the following environment variables will be set, following the format `SM_CHANNEL_[channel_name]`:

`SM_CHANNEL_TRAIN`: A string representing the path to the directory containing data in the 'train' channel
`SM_CHANNEL_VALIDATION`: Same as above, but for the 'validation' channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance. For example, the script that we will run in this notebook is provided as the accompanying file (`abalone.py`).

Because the container imports your training script, always put your training code in a main guard `(if __name__=='__main__':)` so that the container does not inadvertently run your training code at the wrong point in execution.

For more information about training environment variables, please visit https://github.com/aws/sagemaker-training-toolkit.

## Training the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes between few minutes.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __instance_type__: The type of SageMaker instances for training.
* __instance_count__: The number of SageMaker instances for training.
* __sagemaker_session__: The session used to train on Sagemaker.
* __hyperparameters__: A dictionary passed to the train function as hyperparameters.

In [ ]:
hyperparams = {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "reg:squarederror",
        "num_round": "50",
}

instance_type = "ml.m5.2xlarge"
instance_count = 2
output_path = 's3://{}/{}/output'.format(bucket, prefix)
content_type = "parquet"
input_mode = "File"

When using `ShardedByS3Key` SageMaker will distribute the file objects equally to each instance. If your files have different record count you will probably end up with one instance have much more data than others. Therefore our recommendation is to distribute your records equally among your s3 files/objects to make sure each training instance gets same amount of data.

In [ ]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

boto_session = boto3.Session(region_name=region)
session = Session(boto_session=boto_session)
script_path = 'abalone.py'

xgb_estimator = XGBoost(
    source_dir="./",
    entry_point=script_path,
    framework_version='1.5-1', # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=instance_count,
    instance_type=instance_type,
    output_path=output_path,
    input_mode=input_mode
)

train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), 
    distribution='ShardedByS3Key', 
    content_type=content_type)

validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), 
    distribution='FullyReplicated', 
    content_type=content_type)

### Train XGBoost Estimator on abalone data 


Training is as simple as calling `fit` on the Estimator. This will start a SageMaker Training job that will download the data, invoke the entry point code (in the provided script file), and save any model artifacts that the script creates.

In [ ]:
xgb_estimator.fit({'train': train_input, 'validation': validation_input})

## Deploying the XGBoost model

After training, we can use the estimator to create an Amazon SageMaker endpoint – a hosted and managed prediction service that we can use to perform inference.

You can also optionally specify other functions to customize the behavior of deserialization of the input request (`input_fn()`), serialization of the predictions (`output_fn()`), and how predictions are made (`predict_fn()`). The defaults work for our current use-case so we don’t need to define them.

In [ ]:
from sagemaker.serializers import CSVSerializer

predictor = xgb_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.2xlarge",
    serializer=CSVSerializer(),
)

In [ ]:
array = data.iloc[:5, 1:].to_numpy() 
array

In [ ]:
prediction = predictor.predict(array)
prediction

### (Optional) Delete the Endpoint

If you're done with this exercise, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint and avoid any charges from a stray instance being left on.

In [ ]:
predictor.delete_endpoint()

## Orchestration

If you have multiple datasets that may or may not share the same features but go through the same pre-processing and training process, we can orchestrate steps in the process into a workflow. Each time when we prepared a dataset, we only need to kick off the workflow. There are multiple ways to create such workflows:

1. Use [Amazon SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/), a purpose-built CI/CD feature in SageMaker for machine learning. To execute a SageMaker pipeline, you can do it in SageMaker Studio directly OR use [AWS Lambda](https://aws.amazon.com/lambda/) or [Amazon EventBridge](https://aws.amazon.com/eventbridge/) to trigger the execution. For example, you can set up Lambda/EventBridge to trigger the execution of a SageMaker training pipeline when a new training dataset is uploaded to an Amazon S3 bucket.

2. Use [AWS Step Functions](https://aws.amazon.com/step-functions/), which can automate and orchestrate Amazon SageMaker related tasks in an end-to-end workflow. Same as above, a step function can be triggered using Lambda or EventBridge. However, comparing to SageMaker Pipelines, a step function takes more effort to build, and its integration with SageMaker studio and SageMaker other features is worse.

3. Use [Apache Airflow](https://airflow.apache.org/), a popular open source framework for authoring, scheduling, and monitoring multi-stage workflows. Multiple Amazon SageMaker operators are available with Airflow, including model training, hyperparameter tuning, model deployment, and batch transform. This allows you to use the same orchestration tool to manage ML workflows with tasks running on Amazon SageMaker. 